In [166]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
from copy import deepcopy
from numpy.random import randint
import random
import itertools 
from operator import itemgetter

from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

import sys
sys.path.insert(0,'..')

from vf_portalytics.model import PredictionModel

In [190]:
def make_dataset(random_state, n_informative, collumn_names, **kwargs):
    x, y = make_regression(
        
        n_samples=1000, 
        n_features=5,
        noise=0 if random_state == 1 else 10,
        bias=10 if random_state == 1 else 1000,
        n_informative=min(n_informative, 5), 
        random_state=random_state
    )
    x = pd.DataFrame(x)
    x.columns = [name for name in collumn_names]
    x = x.assign(**kwargs)
    x['yearweek'] = randint(1, 54, len(x))
    # pack_type 0: 'Can', 1: 'Bottle'
    x['pack_type'] = random.choices([0, 1], k=len(x))
    
    return x, pd.Series(y)

def make_dict():
    """Creates a dictionary with keys all the combinations between the weeks of the year and the pack types"""
    all_list = [list(range(1, 54)), [0, 1] ]
    keys = list(itertools.product(*all_list))
    values = random.choices(np.linspace(-2.5, 2.5, num=500), k=len(keys))
    return dict(zip(keys, values))

## Generate data and lookup dictionary

In [191]:
collumn_names = ['price', 'promo_week_length', 
                 'yearweek',  'pack_type', 'vol_per_sku']

x1, y1 = make_dataset(1, 5, collumn_names, account_banner='A', product_desc='X')
x2, y2 = make_dataset(2, 3, collumn_names, account_banner='B', product_desc='Y')

# combine into one dataset
total_x = pd.concat([x1, x2], axis=0, ignore_index=True).reset_index(drop=True)
total_y = pd.concat([y1, y2], axis=0, ignore_index=True).reset_index(drop=True)
# Split into train and test
train_index, test_index = train_test_split(total_x.index, random_state=5)
train_x, train_y = total_x.loc[train_index, :], total_y.loc[train_index]
test_x, test_y = total_x.loc[test_index, :], total_y.loc[test_index]

# create dictionary "predicted_market_volumes" - "lookup_dict"
lookup_dict = make_dict()

In [192]:
train_x

53

In [189]:
lookup_dict

{(1, 0): 0.9769539078156311,
 (1, 1): -2.1192384769539077,
 (2, 0): 0.5160320641282565,
 (2, 1): -2.1993987975951903,
 (3, 0): -2.3296593186372747,
 (3, 1): -1.5781563126252505,
 (4, 0): -1.3677354709418839,
 (4, 1): 2.4699398797595187,
 (5, 0): -2.2695390781563125,
 (5, 1): 2.129258517034068,
 (6, 0): -0.8466933867735471,
 (6, 1): 1.5080160320641278,
 (7, 0): -1.588176352705411,
 (7, 1): -0.6763527054108218,
 (8, 0): 1.467935871743487,
 (8, 1): 1.7284569138276549,
 (9, 0): 0.04509018036072154,
 (9, 1): -2.1993987975951903,
 (10, 0): 1.437875751503006,
 (10, 1): -0.5561122244488979,
 (11, 0): -0.18537074148296595,
 (11, 1): -2.379759519038076,
 (12, 0): 1.7885771543086166,
 (12, 1): -0.5761523046092185,
 (13, 0): 2.1593186372745485,
 (13, 1): 0.12525050100200374,
 (14, 0): 0.35571142284569124,
 (14, 1): -1.968937875751503,
 (15, 0): 0.42585170340681344,
 (15, 1): 0.876753507014028,
 (16, 0): 2.1192384769539077,
 (16, 1): 1.2975951903807612,
 (17, 0): 2.1192384769539077,
 (17, 1): -1.73

In [170]:
import pandas as pd
from copy import deepcopy
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin


class FeatureSubsetTransform(BaseEstimator, TransformerMixin):

    def __init__(self, group_cols=None, transformer=None):
        """Build a feature tranformer"""
        self.transformer = transformer
        self.group_cols = group_cols

    def fit(self, X, y=None):
        """Drop the columns that are being used to group the data and fit the transformer"""
        x_in = X.drop([n for n in self.group_cols], axis=1)
        self.transformer = self.transformer.fit(X=x_in[['price']])
        return self

    def transform(self, X):
        x_in = X.drop([n for n in self.group_cols], axis=1)
        # transform the price collumn
        transformed_price = self.transformer.transform(X=x_in[['price']])
        # convert data into initial format
        transformed_price = pd.DataFrame(data=transformed_price, index=x_in.index,
                                     columns=self.transformer.get_feature_names(x_in.columns))
        transformed_price.drop(['1', 'price'], axis=1, inplace=True)
        transformed_x = pd.concat([x_in, transformed_price], axis=1)
        transformed_x[list(self.group_cols)] = X[list(self.group_cols)]
        return transformed_x


class FeatureSubsetModel(BaseEstimator, RegressorMixin):

    def __init__(self, lookup_dict=None, group_cols=None, sub_models=None):
        """
        Build regression model for subsets of feature rows matching particular combination of feature columns.
        """
        self.lookup_dict = lookup_dict
        self.group_cols = group_cols
        self.sub_models = sub_models

    def fit(self, X, y=None):
        """
        Partition the training data, X, into groups for each unique combination of values in
        ``self.group_cols`` columns. For each group, train the appropriate model specified in
        ``self.sub_models``.
        """
        X['predicted_market_volume'] = itemgetter(*zip(X['yearweek'], 
                                                       X['pack_type']))(self.lookup_dict)
        groups = X.groupby(by=list(self.group_cols))
        
        for gp_key, x_group in groups:
            # Find the sub-model for this group key
            gp_model = self.sub_models[gp_key]

            # Drop the feature values for the group columns, since these are same for all rows
            # and so don't contribute anything into the prediction.
            x_in = x_group.drop([n for n in self.group_cols], axis=1)
            y_in = y.loc[x_in.index]
            
            # Fit the submodel with subset of rows and only collumns related to price
            gp_model = gp_model.fit(X=x_in[[col for col in x_in if col.startswith('price')]], y=y_in.values)
            self.sub_models[gp_key] = gp_model
        return self

    def predict(self, X, y=None):
        """
        Same as ``self.fit()``, but call the ``predict()`` method for each submodel and return the results.
        """
        # create a new collumn by checking the lookup_dict
        X['predicted_market_volume'] = itemgetter(*zip(X['yearweek'], 
                                                       X['pack_type']))(self.lookup_dict)
        groups = X.groupby(by=list(self.group_cols))
        results = []

        for gp_key, x_group in groups:
            gp_model = self.sub_models[gp_key]
            x_in = x_group.drop([n for n in self.group_cols], axis=1)
            
            # predict market share only using price related data
            predicted_market_share = gp_model.predict(X=x_in[[col for col in x_in if col.startswith('price')]])
            predicted_market_share = pd.Series(index=x_in.index, data=predicted_market_share)
            
            result = predicted_market_share.mul(
                x_group['predicted_market_volume']).mul(
                x_group['promo_week_length']).div(
                x_group['vol_per_sku']).clip(lower=0)
            
            results.append(result)

        return pd.concat(results, axis=0)

# Create pipeline

In [171]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures


subset_cols = ('account_banner', 'product_desc')
sub_models = {
    ('A', 'X'): LinearRegression(),
    ('B', 'Y'): DecisionTreeRegressor(),
}


pipeline = Pipeline([  
  ('transform', FeatureSubsetTransform(group_cols=subset_cols, transformer=PolynomialFeatures(2))),
  ('estimate', FeatureSubsetModel(lookup_dict=lookup_dict, group_cols=subset_cols, sub_models=sub_models))
])


# Create VF Model Wrapper and Save pipeline

In [172]:
# Note: must use one_hot_encode=False to prevent one-hot encoding of categorical features in input data
model_wrapper = PredictionModel("my_test_model", path='/tmp', one_hot_encode=False)

model_wrapper.model = pipeline
# save feature names (no strictly since all the preprocessing is made being made in the pipeline)
model_wrapper.features = {
    # Grouping features
    'account_banner': [],
    'product_desc': [],
    # other feaures
    'price': [],
    'promo_week_length': [],
    'yearweek': [],
    'pack_type': [],
    'vol_per_sku': [],
}
model_wrapper.target = {'target': []}
model_wrapper.ordered_column_list = sorted(model_wrapper.features.keys())

model_wrapper.model.fit(train_x, train_y)

model_wrapper.save()

# Load Pre-Saved Model 

In [173]:
# Don't specify one_hot_encode here because it will be looked up from the pickle file
saved_model = PredictionModel('my_test_model', path='/tmp')
saved_model.model

Pipeline(memory=None,
         steps=[('transform',
                 FeatureSubsetTransform(group_cols=('account_banner',
                                                    'product_desc'),
                                        transformer=PolynomialFeatures(degree=2,
                                                                       include_bias=True,
                                                                       interaction_only=False,
                                                                       order='C'))),
                ('estimate',
                 FeatureSubsetModel(group_cols=('account_banner',
                                                'product_desc'),
                                    lookup_dict={(0, 0): 1.317635270541082,
                                                 (0, 1): 0.005010020040080221,
                                                 (1, 0): -...
                                                                             fit_

# Test the results

In [174]:
# test for the first group if the pipeline performs what we would like to
groups = train_x.groupby(by=list(subset_cols))
_, train_x = list(groups)[0]

groups = test_x.groupby(by=list(subset_cols))
_, test_x = list(groups)[0]

train_y = train_y.loc[train_x.index]
test_y = test_y.loc[test_x.index]

In [175]:
# predict with pipeline
pipeline_predicted = saved_model.model.predict(test_x)

In [176]:
# drop the columns that declare the group since we use only one group for the test
test_x.drop(list(subset_cols), axis=1, inplace=True)
train_x.drop(list(subset_cols), axis=1, inplace=True)

In [177]:
# transform price collumn
transformer = PolynomialFeatures(2)
transformer.fit(train_x[['price']])

def transform_data(data):
    transformed_price = transformer.transform(data[['price']])
    transformed_price = pd.DataFrame(data=transformed_price, index=data.index,
                                         columns=transformer.get_feature_names(data.columns))
    transformed_price.drop(['1', 'price'], axis=1, inplace=True)
    transformed_x = pd.concat([data, transformed_price], axis=1)
    return transformed_x
train_transformed = transform_data(train_x)
test_transformed = transform_data(test_x)

price_collumns = [col for col in test_transformed if col.startswith('price')]

In [178]:
# predict market share only using price related data
model = LinearRegression().fit(train_transformed[price_collumns], train_y)

predicted_market_share = model.predict(test_transformed[price_collumns])
predicted_market_share = pd.Series(index=test_transformed.index, data=predicted_market_share)

In [179]:
# predict output
test_x['predicted_market_volume'] = itemgetter(*zip(test_x['yearweek'], 
                                                       test_x['pack_type']))(lookup_dict)

directly_predicted = predicted_market_share.mul(
        test_x['predicted_market_volume']).mul(
        test_x['promo_week_length']).div(
        test_x['vol_per_sku']).clip(lower=0)

In [184]:
pd.DataFrame({'directly_predicted': directly_predicted, 'pipeline_predicted': pipeline_predicted})

,directly_predicted,pipeline_predicted
51,0.000000,0.000000
417,0.512737,0.512737
269,0.000000,0.000000
191,0.000000,0.000000
436,0.000000,0.000000
...,...,...
151,1061.389376,1061.389376
629,0.000000,0.000000
574,0.000000,0.000000
886,0.032253,0.032253
